### Debugging input of sensor_agent

This notebook shows how to debug the input and output of model when deployed with `lead.inference.sensor_agent`.

##### Prerequisites
- [Model checkpoints.](https://github.com/ln2697/lead?tab=readme-ov-file#quick-start)

##### Steps

1. Start CARLA

```bash
bash scripts/start_carla.sh
```

2. Run evaluation with `produce_input_log=True`

```bash
LEAD_CLOSED_LOOP_CONFIG="produce_input_log=true" bash scripts/eval_bench2drive.sh
```

3. After the evaluation finished, come back to this notebook

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext ipython_beartype
%beartype

import glob
import os
import sys

carla_root = "./3rd_party/CARLA_0915"  # set your install here
os.environ["CARLA_ROOT"] = carla_root
egg = glob.glob(os.path.join(carla_root, "PythonAPI", "carla", "dist", "carla-*py3*.egg"))[0]
sys.path.append(egg)
sys.path.append(os.path.join(carla_root, "PythonAPI", "carla"))

import torch

from lead.inference.config_open_loop import OpenLoopConfig
from lead.inference.open_loop_inference import OpenLoopInference
from lead.training.config_training import TrainingConfig

Load the configuration and model

In [ ]:
checkpoint_dir = "outputs/checkpoints/tfv6_resnet34"

open_loop_config = OpenLoopConfig()
training_config = TrainingConfig.load_from_file(
    os.path.join(checkpoint_dir, "config.json")
)

open_loop_inference = OpenLoopInference(
    config_training=training_config,
    config_open_loop=open_loop_config,
    model_path=checkpoint_dir,
    device=torch.device("cuda:0"),
)

Load the model input stored on disk.

In [ ]:
input_path = "outputs/local_evaluation/23687/input_log/00001.pth"
model_input = torch.load(input_path)

# Inspect input
(
    model_input["target_point_previous"],
    model_input["target_point"],
    model_input["target_point_next"],
)

Model Inference

In [ ]:
pred = open_loop_inference.forward(model_input)